In [10]:
import pandas as pd


catering_data = pd.read_csv('tbl_catering.csv')
catering_details = pd.read_csv('tbl_catering_details.csv')
catering_products = pd.read_csv('tbl_catering_products.csv')


# # Merge the dataframes on relevant keys
# merged_data = pd.merge(catering_data, catering_details, on=['CATERING_ID', 'CATERING_DET_ID'], how='left')
# merged_data = pd.merge(merged_data, catering_products, left_on='ITEM_DESC', right_on='PRODUCT_NAME', how='left')

# Data Merging
merged_data = pd.merge(tbl_catering, tbl_catering_details, left_on='ID', right_on='CATERING_ID', how='left')
merged_data = pd.merge(merged_data, tbl_catering_products, left_on='ITEM_DESC', right_on='PRODUCT_NAME', how='left')

# Display the merged data to verify
print(merged_data.head())

# Select relevant columns
features = merged_data[['GUEST_COUNT', 'ITEM_DESC']]  # Features: guest count and item description
target = merged_data['QUANTITY']  # Target: quantity of the dish

# Handle any missing values (if necessary)
features['GUEST_COUNT'].fillna(features['GUEST_COUNT'].mean(), inplace=True)
features['ITEM_DESC'].fillna('Unknown', inplace=True)

# Save the merged data if needed for future reference
merged_data.to_csv('merged_catering_data.csv', index=False)


   ID_x DELIVERY_DATE DELIVERY_TIME  DELIVERY_MODE DELIVERY_STATUS  \
0   351     4/24/2021      18:30:00             30              \N   
1   351     4/24/2021      18:30:00             30              \N   
2   351     4/24/2021      18:30:00             30              \N   
3   351     4/24/2021      18:30:00             30              \N   
4   351     4/24/2021      18:30:00             30              \N   

   INVOICE_AMOUNT  TAXES  CATERING_COST  SHOW_PRICE  DISCOUNT  ...  \
0          1525.0    0.0         1525.0           9       0.0  ...   
1          1525.0    0.0         1525.0           9       0.0  ...   
2          1525.0    0.0         1525.0           9       0.0  ...   
3          1525.0    0.0         1525.0           9       0.0  ...   
4          1525.0    0.0         1525.0           9       0.0  ...   

     CATEGORY_NAME  CATEGORY_ID     PRODUCT_NAME  UNIT PRICE_y IS_DAIRY_FREE  \
0     ENTREE - VEG          1.0        Dal Tadka  18.0   130.0             0  

C:\Users\PMYLS\AppData\Local\Temp\ipykernel_17760\3750489057.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['GUEST_COUNT'].fillna(features['GUEST_COUNT'].mean(), inplace=True)
C:\Users\PMYLS\AppData\Local\Temp\ipykernel_17760\3750489057.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['ITEM_DESC'].fillna('Unknown', inplace=True)


In [12]:
from sklearn.impute import SimpleImputer

# Impute missing values for numerical features
numerical_imputer = SimpleImputer(strategy='mean')
features['GUEST_COUNT'] = numerical_imputer.fit_transform(features[['GUEST_COUNT']])

# If the target has NaN values, drop those rows
target = target.dropna()
features = features.loc[target.index]  # Align features with target

# One-hot encode the item description
features = pd.get_dummies(features, columns=['ITEM_DESC'], drop_first=True)


C:\Users\PMYLS\AppData\Local\Temp\ipykernel_17760\2797457619.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['GUEST_COUNT'] = numerical_imputer.fit_transform(features[['GUEST_COUNT']])


In [24]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
import joblib

# One-hot encode the item description
column_transformer = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), ['ITEM_DESC', 'SPICE_LEVEL']),
        ('num', StandardScaler(), ['QUANTITY', 'PRICE_x'])  # Adjust to match your DataFrame
    ]
)


# Create the pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessor', column_transformer),
    ('model', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Train the model
model_pipeline.fit(X_train, y_train)

# Save the trained model
joblib.dump(model_pipeline, 'dish_quantity_predictor.pkl')


ValueError: A given column is not a column of the dataframe

In [21]:
print(merged_data.columns)


Index(['ID_x', 'DELIVERY_DATE', 'DELIVERY_TIME', 'DELIVERY_MODE',
       'DELIVERY_STATUS', 'INVOICE_AMOUNT', 'TAXES', 'CATERING_COST',
       'SHOW_PRICE', 'DISCOUNT', 'GUEST_COUNT', 'PER_PERSON_RATE',
       'LIVE_STALL_CHARGE', 'SETUP_CHARGE', 'EVENT_FACILITY_FEE',
       'SERVER_CHARGE', 'GRATUITY', 'CATERING_STATUS', 'CATERING_LOCATION',
       'ID_y', 'CATERING_ID', 'CATERING_DET_ID', 'ITEM_DESC', 'SPICE_LEVEL',
       'QUANTITY', 'PRICE_x', 'DAIRY', 'GLUTEN', 'NUTS', 'VEGAN', 'COMMENTS',
       'LNL_LOCATION', 'CREATED_DTTM', 'CREATED_BY_ID', 'UPDATED_DTTM',
       'UPDATED_BY_ID', 'PRODUCT_ID', 'CATEGORY_NAME', 'CATEGORY_ID',
       'PRODUCT_NAME', 'UNIT', 'PRICE_y', 'IS_DAIRY_FREE', 'IS_GLUTEN_FREE',
       'IS_NUT_FREE', 'IS_VEGAN', 'EXCLUDE_ITEM'],
      dtype='object')


In [25]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
import joblib

# Verify the exact column names in your DataFrame
print(features.columns)

# One-hot encode the product name and standardize numeric columns
column_transformer = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), ['PRODUCT_NAME', 'SPICE_LEVEL']),  # Adjust to match your DataFrame
        ('num', StandardScaler(), ['QUANTITY', 'PRICE_x'])
    ]
)

# Create the pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessor', column_transformer),
    ('model', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Train the model
model_pipeline.fit(X_train, y_train)

# Save the trained model
joblib.dump(model_pipeline, 'dish_quantity_predictor.pkl')


Index(['GUEST_COUNT', 'ITEM_DESC_0', 'ITEM_DESC_AAM PANNA', 'ITEM_DESC_AAMRAS',
       'ITEM_DESC_ALASANDA VADA', 'ITEM_DESC_ALFREDO PASTA',
       'ITEM_DESC_ALOO BONDA', 'ITEM_DESC_ALOO CHUKKA',
       'ITEM_DESC_ALOO GOBI FRY', 'ITEM_DESC_ALOO GOBI MASALA',
       ...
       'ITEM_DESC_WATER BOTTLE - 8 OZ', 'ITEM_DESC_WATERMELON MOCKTAILS',
       'ITEM_DESC_WELCOME DRINK',
       'ITEM_DESC_WELCOME DRINK  - FRUIT PUNCH / MANGO LASSI / STRAWBERRY LASSI',
       'ITEM_DESC_WHISKY - GLENFIDDICH', 'ITEM_DESC_WHITE RICE',
       'ITEM_DESC_WHITE WINE', 'ITEM_DESC_YAM FRY', 'ITEM_DESC_YOGURT',
       'ITEM_DESC_ZAFFRANI PULAO'],
      dtype='object', length=1214)


ValueError: A given column is not a column of the dataframe